In [97]:
import boto3
import io
import re
import numpy as np
import os
import xml.etree.ElementTree as ET 
from google.cloud import vision
import requests
from nltk.corpus import stopwords as stpword
from nltk.stem import WordNetLemmatizer
import json
from nltk.corpus import stopwords as stpword
from nltk.stem import WordNetLemmatizer
from utilities.image_description_scoring import matching_label_score
from time import time
from utilities.pythonDB import writeToDB, recordsExists
from utilities.score_computors import jaccard_similarity

In [98]:
def get_mapped(path):
    key_map, dict_idx = {}, 0
    for r, _, f in os.walk(path):
        for file in f:
            if '.JPG' in file or '.jpg' in file:
                key_map[dict_idx] = file
                dict_idx += 1
    idxs = list(key_map.keys())          
    shuffled_idxs = np.random.randint(0, len(idxs), len(idxs))
    return key_map, shuffled_idxs

def extract_clean_str(original_str):
    original_str = original_str.replace('<br/>', '')
    return re.sub('[^a-zA-Z0-9]+', ' ', original_str, flags=re.UNICODE).lower().strip().replace(' ', '')

def fetch_from_xml_natocr(path, file):
    tree = ET.parse(os.path.join(path, file)).getroot()  
    return extract_clean_str(''.join([e[0].text for e in tree.iter(tag='object')]))

In [99]:
class RegonizeText():
    def __init__(self, base, file): 
        with io.open(os.path.join(base, file), 'rb') as image_file:
            content = (image_file.read())
        self.content = content
        self.base, self.actual_str, self.detected_str = base, '', ''
        self.file = '_'.join(file.split('_')[-2:])
        self.severity, self.noise_type = file.split('_')[-3], '_'.join(file.split('_')[1:-3])
        
        self.label = ''
        self.start = time()
        
    def return_function(self, name):
        dataset = dir_path.split('/')[1] 
        self.label = name + '-OCR-noisy'
        if recordsExists(self.file, dataset, self.label, self.severity, self.noise_type):
            return
        
        status, score =  getattr(self, 'if_' + name)()
        compute_time = time() - self.start
        bag = (self.file, dataset, self.label, self.severity, self.noise_type, status, score, compute_time, self.actual_str, self.detected_str)
        print (bag)
        #writeToDB(bag)
    def compute_ground_truth(self):
        ''' 
            Status = {-1: 'XML File Missing', -2: 'Error in XML', -3: 'Ground Truth Empty', -4: 'API Error', 0: 'All Correct'} 
            Return = Ground Truth, Status
        
        '''
        xml_path = '/'.join(self.base.split('/')[:2]) + '/OCR'
        xml_file = self.file[:len(self.file)-4] + '.xml' 
        
        if not os.path.exists(os.path.join(xml_path, self.file)):
            return '', -1
        try:
            ground_truth_str = fetch_from_xml_natocr(xml_path, xml_file)
        except: 
            return '', -2
        
        if len(ground_truth_str) == 0:
            return '', -3
        else:
            self.actual_str = ground_truth_str
            return ground_truth_str, 0

    def if_aws(self):
        ground_truth, xml_status = self.compute_ground_truth()
        if xml_status != 0: return xml_status, 0
        
        try:
            imgobj = {'Bytes': self.content}
            client=boto3.client('rekognition', region_name='us-east-1')
            response=client.detect_text(Image=imgobj)
            detected_str = extract_clean_str(''.join([txt['DetectedText'] for txt in response['TextDetections'] \
                                                   if txt['Type']=='WORD']))
            self.detected_str = detected_str
            print ("I am here 1 ")
            score = jaccard_similarity(detected_str, ground_truth)
            print ("I am here 2 ")
            return 0, score
        except:
            return -4, 0

In [100]:
dir_path = 'datasets/neocr_dataset/noises/'
dict_files, files_idx = get_mapped(dir_path)
shuffled_idx = np.random.randint(0, len(files_idx), len(files_idx))[:1]

In [101]:
for idx in shuffled_idx:
    file_name = dict_files.get(idx)
    print ("Iterating for File Name = {}".format(file_name))
    RegonizeText(dir_path, file_name).return_function('aws')

Iterating for File Name = add_shot_noise_4_img_941768284.jpg
I am here 1 
I am here 2 
('img_941768284.jpg', 'neocr_dataset', 'aws-OCR-noisy', 0, 0.2358490566037736, 3.085570812225342, 'erlangerbergkirchweihdonnerstag27052010familienundkindertagermigtefahrundeintrittspreisebis2000uhrbetriebeerikaeuropa', 'erlangerbergkirchweihetriebedonnerstag27052010familienundkindertagintefabrundeintritispreisebis2000uhrerik')


In [105]:
dir_path = 'datasets/neocr_dataset/noises/'
dict_files, files_idx = get_mapped(dir_path)
shuffled_idx = np.random.randint(0, len(files_idx), len(files_idx))[:2000]

In [135]:
file = dict_files.get(0)
file

'add_impulse_noise_4_img_297611753.jpg'

In [150]:
file = 'random_image_eraser_000000491277.jpg'
file_name, severity, noise_type = '', '', ''
if 'img' in file:
    file_name = '_'.join(file.split('_')[-2:])
    if 'eraser' in file:
        severity, noise_type = -1, '_'.join(file.split('_')[1:-2])
    else:
        severity, noise_type = file.split('_')[-3], '_'.join(file.split('_')[1:-3])
else:
    file_name = '_'.join(file.split('_')[-1:])
    if 'eraser' in file:
        severity, noise_type = -1, '_'.join(file.split('_')[1:-1])
    else:
        severity, noise_type = file.split('_')[-2], '_'.join(file.split('_')[1:-2])
    
file_name, severity, noise_type

('000000491277.jpg', -1, 'image_eraser')

In [124]:
file = 'perform_swirl_transformation_4_000000276735.jpg'

In [121]:
file_name = '_'.join(file.split('_')[-1:])
file_name

'000000427438.jpg'

In [122]:
severity, noise_type = file.split('_')[-2], '_'.join(file.split('_')[1:-2])
severity, noise_type

('4', 'gaussian_noise')